In [ ]:
!pip install --upgrade pip
!pip install tensorflow
import os

# Defining each of these directories

In [ ]:
train_dir = os.path.join('../input/railway-track-fault-detection-dataset1-rail/Dataset 1_Rail/Train')
validation_dir = os.path.join('../input/railway-track-fault-detection-dataset1-rail/Dataset 1_Rail/Validiation')
test_dir = os.path.join('../input/railway-track-fault-detection-dataset1-rail/Dataset 1_Rail/Test')

# Directory with our training defective/nondefective pictures
train_defective_dir = os.path.join('../input/railway-track-fault-detection-dataset1-rail/Dataset 1_Rail/Train/Defective')
train_nondefective_dir = os.path.join('../input/railway-track-fault-detection-dataset1-rail/Dataset 1_Rail/Train/Non Defective')
# Directory with our validation defective/nondefective pictures
validation_defective_dir = os.path.join('../input/railway-track-fault-detection-dataset1-rail/Dataset 1_Rail/Validiation/Defective')
validation_nondefective_dir = os.path.join('../input/railway-track-fault-detection-dataset1-rail/Dataset 1_Rail/Validiation/Non Defective')

test_defective_dir = os.path.join('../input/railway-track-fault-detection-dataset1-rail/Dataset 1_Rail/Test/Defective')
test_nondefective_dir = os.path.join('../input/railway-track-fault-detection-dataset1-rail/Dataset 1_Rail/Test/Non Defective')

**Now, let's see what the filenames look like in the training directories:**

In [ ]:
train_defective_fnames = os.listdir(train_defective_dir )
train_nondefective_fnames = os.listdir( train_nondefective_dir)

print(train_defective_fnames[:20])
print(train_nondefective_fnames[:20])

In [ ]:
print('total training defective images :', len(os.listdir(train_defective_dir)))
print('total training non-defective images :', len(os.listdir(train_nondefective_dir)))


print('total validation defective images :', len(os.listdir( validation_defective_dir ) ))
print('total validation non-defective images :', len(os.listdir( validation_nondefective_dir) ))

print('total test defective images :', len(os.listdir( validation_defective_dir ) ))
print('total test non-defective images :', len(os.listdir( validation_nondefective_dir) ))

dimensionality**Now let's take a look at a few pictures to get a better sense of what they look like. First, configure the matplot parameters:**

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Parameters for our graph; we'll output images in a 10x10 configuration
nrows = 10
ncols = 10

# Index for iterating over images
pic_index = 0

In [ ]:
# Set up matplotlib fig, and size it to fit 4x4 pics
fig = plt.gcf()
fig.set_size_inches(ncols * 5, nrows * 5)

pic_index += 10
next_defective_pix = [os.path.join(train_defective_dir, fname) 
                for fname in train_defective_fnames[pic_index-10:pic_index]]
next_nondefective_pix = [os.path.join(train_nondefective_dir, fname) 
                for fname in train_nondefective_fnames[pic_index-10:pic_index]]


for i, img_path in enumerate(next_defective_pix+next_nondefective_pix):
  # Set up subplot; subplot indices start at 1
  sp = plt.subplot(nrows, ncols, i + 1)
  sp.axis('Off') # Don't show axes (or gridlines)

  img = mpimg.imread(img_path)
  plt.imshow(img)

plt.show()

In [ ]:
import tensorflow as tf

**The "output shape" column shows how the size of your feature map evolves in each successive layer. The convolution layers reduce the size of the feature maps by a bit due to padding, and each pooling layer halves the dimensions.**

#  Building the Model

In [ ]:
#MUL 1 - Inception - ST
#import keras
# from keras.applications import InceptionV3
# from keras.applications import Xception
from tensorflow.keras.applications import InceptionResNetV2
#from keras.applications import V2
#model = VGG16()
from  tensorflow.keras.models import Model
# from keras.layers import concatenate
from  tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input, Embedding, LSTM, GRU, Reshape, Concatenate,Bidirectional, Conv2D, MaxPooling2D, UpSampling2D, Flatten
#from  tensorflow.keras.applications.mobilenet import preprocess_input

from  tensorflow.keras.layers import GaussianNoise
input_img = Input(shape = (600, 600, 3))
#f1_x = tensorflow.keras.layers.Flatten()
#encoder
f1_x = Conv2D(128, (6, 6), activation='relu', kernel_initializer='he_normal', padding='valid')(input_img) #28 x 28 x 32
tf.keras.layers.BatchNormalization()
tf.keras.layers.Dropout(rate=0.4)
f1_x = MaxPooling2D(pool_size=(2, 2))(f1_x) #14 x 14 x 32
tf.keras.layers.BatchNormalization()
tf.keras.layers.Dropout(rate=0.35)
f1_x = Conv2D(64, (4, 4), activation='relu', kernel_initializer='he_normal', padding='valid')(f1_x) #14 x 14 x 64
tf.keras.layers.BatchNormalization()
tf.keras.layers.Dropout(rate=0.33)
f1_x = MaxPooling2D(pool_size=(2, 2))(f1_x) #7 x 7 x 64
tf.keras.layers.BatchNormalization()
tf.keras.layers.Dropout(rate=0.3)
f1_x = Conv2D(64, (4, 4), activation='relu', padding='valid',kernel_initializer='he_normal')(f1_x) #7 x 7 x 128 (small and thick)
tf.keras.layers.BatchNormalization()
tf.keras.layers.Dropout(rate=0.4)
f1_x = MaxPooling2D(pool_size=(2, 2))(f1_x)
tf.keras.layers.BatchNormalization()
tf.keras.layers.Dropout(rate=0.49)
#decoder
f1_x = Conv2D(64, (6, 6), activation='relu', padding='valid',kernel_initializer='he_normal')(f1_x) #7 x 7 x 128
tf.keras.layers.BatchNormalization()
tf.keras.layers.Dropout(rate=0.36)
f1_x = UpSampling2D((2,2))(f1_x) # 14 x 14 x 128
tf.keras.layers.BatchNormalization()
tf.keras.layers.Dropout(rate=0.38)
f1_x = Conv2D(128, (6, 6), activation='relu', padding='valid',kernel_initializer='he_normal')(f1_x) # 14 x 14 x 64
tf.keras.layers.BatchNormalization()
#dropout=0.2
f1_x = UpSampling2D((2,2))(f1_x) # 28 x 28 x 64
tf.keras.layers.BatchNormalization()
tf.keras.layers.Dropout(rate=0.37)
f1_x = Conv2D(1, (6, 6), activation='sigmoid', padding='valid', kernel_initializer='he_normal')(f1_x) # 28 x 28 x 1
tf.keras.layers.BatchNormalization()
tf.keras.layers.BatchNormalization()

tf.keras.layers.Dropout(rate=0.5)
#f1_x = MinPooling2D(pool_size=(3, 3))(f1_x)     
# f1_base = Xception(weights='imagenet', include_top=False, input_shape=(450,450,3))
# f1_base = InceptionV3(weights='imagenet', include_top=False, input_shape=(299,299,3))
f1_base = InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(600,600,3))
tf.keras.layers.Dropout(rate=0.45)
tf.keras.layers.BatchNormalization()
tf.keras.layers.Dropout(rate=0.5)
tf.keras.layers.BatchNormalization()
f1_x = f1_base.output
tf.keras.layers.BatchNormalization()
f1_x = GlobalAveragePooling2D()(f1_x)
tf.keras.layers.BatchNormalization()
tf.keras.layers.Dropout(rate=0.4)
#f1_x = Conv2D(128, (3, 3), activation='relu', padding='same')(f1_x) #7 x 7 x 128
#f1_x = MaxPooling2D(pool_size=(2, 2))(f1_x) 
f1_x = Reshape([1,1536])(f1_x)  
f1_x = Bidirectional(tf.keras.layers.LSTM(2900, 
return_sequences=False, 
dropout=0.55),
input_shape=[1,1024],
merge_mode='concat')(f1_x)
tf.keras.layers.BatchNormalization()
#f1_x = Conv2D(128, (3, 3), activation='relu', padding='same')(f1_x) #7 x 7 x 128
#f1_x = MaxPooling2D(pool_size=(2, 2))(f1_x) 
#Regularization with noise
f1_x = GaussianNoise(0.20)(f1_x)
tf.keras.layers.BatchNormalization()
f1_x = Dense(100,activation='relu', kernel_initializer='he_normal',kernel_regularizer=tf.keras.regularizers.l1_l2(0.01),kernel_constraint=tf.keras.constraints.max_norm(1.))(f1_x)
tf.keras.layers.BatchNormalization()
tf.keras.layers.Dropout(rate=0.5)
tf.keras.layers.BatchNormalization()
f1_x = Dense(100,activation='relu', kernel_initializer='he_normal',kernel_constraint=tf.keras.constraints.max_norm(1.))(f1_x)
tf.keras.layers.Dropout(rate=0.58)
tf.keras.layers.BatchNormalization()
tf.keras.layers.BatchNormalization()
tf.keras.layers.BatchNormalization()
tf.keras.layers.Dropout(rate=0.46)
tf.keras.layers.BatchNormalization()
#he_avg_int = keras.initializers.VarianceScaling(scale=2, mode = 'fan_avg', distribution='uniform')
#f1_x = Conv2D(64, (3, 3), activation='relu', padding='same')(f1_x)
f1_x = Dense(1, activation='sigmoid', kernel_initializer='he_normal')(f1_x)
model_1 = Model(inputs=[f1_base.input],outputs=[f1_x])
print(model_1.summary())

In [ ]:
## fix for multi_gpu_model prediction time longer
from tensorflow.keras.layers import Lambda, concatenate
from tensorflow.keras import Model
import tensorflow as tf

def multi_gpu_model(model, gpus):
    if isinstance(gpus, (list, tuple)):
        num_gpus = len(gpus)
        target_gpu_ids = gpus
    else:
        num_gpus = gpus
        target_gpu_ids = range(num_gpus)

    def get_slice(data, i, parts):
        shape = tf.shape(data)
        batch_size = shape[:1]
        input_shape = shape[1:]
        step = batch_size // parts
        if i == num_gpus - 1:
            size = batch_size - step * i
        else:
            size = step
        size = tf.concat([size, input_shape], axis=0)
        stride = tf.concat([step, input_shape * 0], axis=0)
        start = stride * i
        return tf.slice(data, start, size)

    all_outputs = []
    for i in range(len(model.outputs)):
        all_outputs.append([])

    # Place a copy of the model on each GPU,
    # each getting a slice of the inputs.
    for i, gpu_id in enumerate(target_gpu_ids):
        with tf.device('/gpu:%d' % gpu_id):
            with tf.name_scope('replica_%d' % gpu_id):
                inputs = []
                # Retrieve a slice of the input.
                for x in model.inputs:
                    input_shape = tuple(x.get_shape().as_list())[1:]
                    slice_i = Lambda(get_slice,
                                   output_shape=input_shape,
                                   arguments={'i': i,
                                              'parts': num_gpus})(x)
                    inputs.append(slice_i)

                # Apply model on slice
                # (creating a model replica on the target device).
                outputs = model(inputs)
                if not isinstance(outputs, list):
                    outputs = [outputs]

                # Save the outputs for merging back together later.
                for o in range(len(outputs)):
                    all_outputs[o].append(outputs[o])

    # Merge outputs on CPU.
    with tf.device('/cpu:0'):
        merged = []
        for name, outputs in zip(model.output_names, all_outputs):
            merged.append(concatenate(outputs,
                                    axis=0, name=name))
        return Model(model.inputs, merged)

In [ ]:
from tensorflow.keras.callbacks import Callback
import pickle
import sys

#Stop training on val_acc
class EarlyStoppingByAccVal(Callback):
    def __init__(self, monitor='val_acc', value=0.00001, verbose=1):
        super(Callback, self).__init__()
        self.monitor = monitor
        self.value = value
        self.verbose = verbose

    def on_epoch_end(self, epoch, logs={}):
        current = logs.get(self.monitor)
        if current is None:
            warnings.warn("Early stopping requires %s available!" % self.monitor, RuntimeWarning)

        if current >= self.value:
            if self.verbose > 0:
                print("Epoch %05d: early stopping" % epoch)
            self.model.stop_training = True

#Save large model using pickle formate instead of h5            
class SaveCheckPoint(Callback):
    def __init__(self, model, dest_folder):
        super(Callback, self).__init__()
        self.model = model
        self.dest_folder = dest_folder
        
        #initiate
        self.best_val_acc = 0
        self.best_val_loss = sys.maxsize #get max value
          
    def on_epoch_end(self, epoch, logs={}):
        val_acc = logs['val_acc']
        val_loss = logs['val_loss']

        if val_acc > self.best_val_acc:
            self.best_val_acc = val_acc
            
            # Save weights in pickle format instead of h5
            print('\nSaving val_acc %f at %s' %(self.best_val_acc, self.dest_folder))
            weigh= self.model.get_weights()

            #now, use pickle to save your model weights, instead of .h5
            #for heavy model architectures, .h5 file is unsupported.
            fpkl= open(self.dest_folder, 'wb') #Python 3
            pickle.dump(weigh, fpkl, protocol= pickle.HIGHEST_PROTOCOL)
            fpkl.close()
            
#             model.save('tmp.h5')
        elif val_acc == self.best_val_acc:
            if val_loss < self.best_val_loss:
                self.best_val_loss=val_loss
                
                # Save weights in pickle format instead of h5
                print('\nSaving val_acc %f at %s' %(self.best_val_acc, self.dest_folder))
                weigh= self.model.get_weights()

                #now, use pickle to save your model weights, instead of .h5
                #for heavy model architectures, .h5 file is unsupported.
                fpkl= open(self.dest_folder, 'wb') #Python 3
                pickle.dump(weigh, fpkl, protocol= pickle.HIGHEST_PROTOCOL)
                fpkl.close()

In [ ]:
ls -l

In [ ]:
#Non-Groups
#Split training and validation
#Using Expert Data

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, CSVLogger, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adamax
# from keras.utils import multi_gpu_model

import time, os
from math import ceil
import multiprocessing

train_datagen = ImageDataGenerator(
    rescale = 1./255,
    width_shift_range=0.15,
    height_shift_range=0.2,
    zoom_range=0.35,
    channel_shift_range=15,##
    fill_mode='nearest'
    #preprocessing_function=preprocess_input,
)

test_datagen = ImageDataGenerator(
    rescale = 1./255
    #preprocessing_function = preprocess_input
)

NUM_GPU = 1
batch_size = 8
train_set = train_datagen.flow_from_directory('../input/railway-track-fault-detection-dataset1-rail/Dataset 1_Rail/Train',
                                                 target_size = (600, 600),
                                                 batch_size = batch_size,
                                                 class_mode = 'binary',
                                                 shuffle=True,
                                                 seed=10,
#                                                  subset="training"
                                              )

valid_set = test_datagen.flow_from_directory('../input/railway-track-fault-detection-dataset1-rail/Dataset 1_Rail/Validiation',
                                                 target_size = (600,600),
                                                 batch_size = batch_size,
                                                 class_mode = 'binary',
                                                 shuffle=True,
                                                 seed=10,
#                                                  subset="validation"
                                             )

model_txt = 'st'
# Helper: Save the model.
savedfilename = os.path.join('checkpoints', 'Railway_IRESNET_LSTM.hdf5')

checkpointer = ModelCheckpoint(savedfilename,
                          monitor='val_acc', verbose=1, 
                          save_best_only=True, mode='max',save_weights_only=True)########

# Helper: TensorBoard
tb = TensorBoard(log_dir=os.path.join('svhn_output', 'logs', model_txt))

# Helper: Save results.
timestamp = time.time()
csv_logger = CSVLogger(os.path.join('svhn_output', 'logs', model_txt + '-' + 'training-' + \
    str(timestamp) + '.log'))

earlystopping = EarlyStoppingByAccVal(monitor='val_acc', value=0.9900, verbose=1)

#Using multiple models if more than 1 GPU
if NUM_GPU != 1:
    model_mul = multi_gpu_model(model_1, gpus=NUM_GPU)
else:
    model_mul = model_1
    
epochs = 50 ##!!!
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                              patience=5, min_lr=0.001)
#decay = reduce_lr/epochs
optimizer = Adamax()

model_mul.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])

step_size_train=ceil(train_set.n/train_set.batch_size)
step_size_valid=ceil(valid_set.n/valid_set.batch_size)
# step_size_test=ceil(testing_set.n//testing_set.batch_size)
tf.keras.backend.set_learning_phase(1)

result = model_mul.fit_generator(
    generator = train_set, 
    steps_per_epoch = step_size_train,
    validation_data = valid_set,
    validation_steps = step_size_valid,
    shuffle=True,
    epochs=epochs,
    callbacks=[reduce_lr],
    #callbacks=[tb, csv_logger, checkpointer, earlystopping],        
    verbose=1)
tf.keras.backend.set_learning_phase(0)

In [ ]:
import numpy as np
print(result.history['val_accuracy'])
print(np.mean(result.history['val_accuracy']))
print(np.max(result.history['val_accuracy']))
print(result.history['accuracy'])
print(np.mean(result.history['accuracy']))
print(np.max(result.history['accuracy']))

In [ ]:
import matplotlib.pyplot as plt
history = result
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.grid()
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
import cv2
import numpy as np
img = cv2.imread('../input/railwaytrackv4/Dataset _ Railway Track Fault Detection-20210713T183411Z-001/Dataset _ Railway Track Fault Detection/Test/Defective/IMG_20201114_101200.jpg')
plt.imshow(img)
img = cv2.resize(img,(400,400))
img = np.reshape(img,[400,400,3])

pred_value = model_mul.predict_generator(img)

print(pred_value)
if pred_value>0.5:
    print("This Railway track has no fault")
else:
    print("This Railway track has fault")

In [ ]:
import cv2
import numpy as np
img = cv2.imread('../input/railwaytrackv4/Dataset _ Railway Track Fault Detection-20210713T183411Z-001/Dataset _ Railway Track Fault Detection/Test/Defective/aug_prefix_0_2872.jpg')
plt.imshow(img)
img = cv2.resize(img,(650,650))
img = np.reshape(img,[1,650,650,3])

classes = model_mul.predict(img)

print(classes)
if classes>0.5:
    print("This Railway track has no fault")
else:
    print("This Railway track has fault")

In [ ]:
import cv2
import numpy as np
img = cv2.imread('../input/railwaytrackv4/Dataset _ Railway Track Fault Detection-20210713T183411Z-001/Dataset _ Railway Track Fault Detection/Test/Defective/aug_prefix_0_2958.jpg')
plt.imshow(img)
img = cv2.resize(img,(650,650))
img = np.reshape(img,[1,650,650,3])

pred_value = model_mul.predict_generator(img)

print(pred_value)
if pred_value>0.5:
    print("This Railway track has no fault")
else:
    print("This Railway track has fault")

In [ ]:
import cv2
import numpy as np
img = cv2.imread('../input/railwaytrackv4/Dataset _ Railway Track Fault Detection-20210713T183411Z-001/Dataset _ Railway Track Fault Detection/Test/Non Defective/IMG_20201114_102909.jpg')
plt.imshow(img)
img = cv2.resize(img,(650,650))
img = np.reshape(img,[1,650,650,3])

classes = model_mul.predict(img)

print(classes)
if classes>0.5:
    print("This Railway track has no fault")
else:
    print("This Railway track has fault")

In [ ]:
import cv2
import numpy as np
img = cv2.imread('../input/railwaytrackv4/Dataset _ Railway Track Fault Detection-20210713T183411Z-001/Dataset _ Railway Track Fault Detection/Test/Non Defective/aug_prefix_0_3373.jpg')
plt.imshow(img)
img = cv2.resize(img,(650,650))
img = np.reshape(img,[1,650,650,3])

pred_value = model_mul.predict_generator(img)

print(pred_value)
if pred_value>0.5:
    print("This Railway track has no fault")
else:
    print("This Railway track has fault")